In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## LangGraph Example 1


In [126]:
from langgraph.graph import Graph
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# Node 1
def get_prompt(state: dict) -> dict:
    question = state["question"]
    prompt = question + "\nNOTE: Provide the answer only, do the calculation in your mind"
    return {"prompt": prompt}


# Node 2
def llm_response(state: dict) -> dict:
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
    prompt = state["prompt"]
    chain = llm | StrOutputParser()
    answer = chain.invoke(prompt)
    return {"answer": answer}


workflow = Graph()

workflow.add_node("get_prompt", get_prompt)
workflow.add_node("response", llm_response)

workflow.set_entry_point("get_prompt")
workflow.add_edge("get_prompt", "response")
workflow.set_finish_point("response")

app = workflow.compile()
app.invoke({"question": "What is 2 + 10?"})

{'answer': '12'}

## LangGraph Example 2


In [127]:
from typing import TypedDict
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


class GraphState(TypedDict):
    question: str
    prompt: str
    answer: str


# Node 1
def get_prompt(state: GraphState) -> GraphState:
    question = state["question"]
    prompt = question + "\nNOTE: Provide the answer only, do the calculation in your mind"
    return GraphState(prompt=prompt)


# Node 2
def llm_response(state: GraphState) -> GraphState:
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
    prompt = state["prompt"]
    chain = llm | StrOutputParser()
    answer = chain.invoke(prompt)
    return GraphState(answer=answer)


workflow = StateGraph(GraphState)

workflow.add_node("get_prompt", get_prompt)
workflow.add_node("response", llm_response)

workflow.set_entry_point("get_prompt")
workflow.add_edge("get_prompt", "response")
workflow.set_finish_point("response")

app = workflow.compile()
app.invoke({"question": "What is 2 + 10?"})

{'question': 'What is 2 + 10?',
 'prompt': 'What is 2 + 10?\nNOTE: Provide the answer only, do the calculation in your mind',
 'answer': '12'}

## LangGraph Example 3


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate


# Analysis Agent: classifies input into 'code' or 'general'
def analyze_question(state):
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
    prompt = PromptTemplate.from_template(
        """
        You are an agent tasked with categorizing a user question.
        Question: {input}
        Only reply "code" if about programming or technical topics; if not, reply "general".
        Your answer (code/general):
    """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    decision = response.content.strip().lower()
    return {"decision": decision, "input": state["input"]}


# Code Agent: Detailed programming-centric answers
def answer_code_question(state):
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
    prompt = PromptTemplate.from_template(
        """
        You are a skilled software developer. Answer thoroughly with clear, structured steps:
        {input}
    """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"output": response}


# General Agent: Concise answers on general topics
def answer_generic_question(state):
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
    prompt = PromptTemplate.from_template(
        """
        You are a helpful assistant. Provide a clear, concise answer:
        {input}
    """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"output": response}

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict


# Defining our shared state for clarity
class AgentState(TypedDict):
    input: str
    output: str
    decision: str


def create_graph():
    workflow = StateGraph(AgentState)

    # Adding our agent nodes
    workflow.add_node("analyze", analyze_question)
    workflow.add_node("code_agent", answer_code_question)
    workflow.add_node("generic_agent", answer_generic_question)

    # Conditions to select the next agent
    workflow.add_conditional_edges(
        "analyze", lambda state: state["decision"], {"code": "code_agent", "general": "generic_agent"}
    )

    # Define workflow entry and ending points
    workflow.set_entry_point("analyze")
    workflow.add_edge("code_agent", END)
    workflow.add_edge("generic_agent", END)

    return workflow.compile()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Keep API keys secure using environment variables or secure storage. Never directly embed keys in your code.


class UserInput(TypedDict):
    input: str
    continue_conversation: bool


# Get user input
def get_user_input(state: UserInput) -> UserInput:
    user_input = input("\nAsk your question (type 'q' to quit): ")
    return {"input": user_input, "continue_conversation": user_input.lower() != "q"}


# Process question through our LangGraph workflow
def process_question(state: UserInput):
    graph = create_graph()
    result = graph.invoke({"input": state["input"]})
    print("\n--- Here's your answer ---")
    print(result["output"])
    return state


# Create conversation loop graph
def create_conversation_graph():
    workflow = StateGraph(UserInput)
    workflow.add_node("get_input", get_user_input)
    workflow.add_node("process_question", process_question)
    workflow.add_conditional_edges(
        "get_input",
        lambda x: "continue" if x["continue_conversation"] else "stop",
        {"continue": "process_question", "stop": END},
    )
    workflow.add_edge("process_question", "get_input")
    workflow.set_entry_point("get_input")
    return workflow.compile()


# Run the main interactive conversation
def main():
    conversation_graph = create_conversation_graph()
    conversation_graph.invoke({"input": "", "continue_conversation": True})


if __name__ == "__main__":
    main()


--- Here's your answer ---
content='CI/CD pipeline is a fundamental concept in modern software development, particularly in practices like DevOps and Agile development. CI stands for Continuous Integration, and CD stands for Continuous Delivery and/or Continuous Deployment. These methodologies are designed to improve software development processes, making them faster, more efficient, and reducing the risk of errors. Here’s a detailed breakdown of each component and how they form a CI/CD pipeline:\n\n### Continuous Integration (CI)\n\n**Purpose:** The primary goal of Continuous Integration is to integrate code changes into a shared repository frequently, ideally several times a day. This practice helps in detecting errors quickly, and improving software quality.\n\n**Steps Involved:**\n1. **Version Control:** Developers merge their changes back to the main branch as often as possible. Each merge is verified by creating a build and running automated tests against the build.\n2. **Automa

# Weekly planner


In [ ]:
from typing import Dict, Union
from langchain_core.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain_openai import ChatOpenAI
import json

# Tool 1: Structured with auto-schema
from langchain_core.tools import tool


@tool
def list_week_days(goal: str) -> dict:
    """Break a goal into a plan for each day of the week."""
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return {day: f"{goal} - Step {i + 1}" for i, day in enumerate(days)}


# Tool 2: Manual (no Pydantic args_schema)
def format_as_todo_list_func(day_plan: Union[Dict[str, str], str]) -> str:
    """Format a daily plan dictionary into a weekly to-do list."""
    todo = ""

    # Handle the case when day_plan is passed as a string (likely JSON)
    if isinstance(day_plan, str):
        try:
            day_plan = json.loads(day_plan)
        except json.JSONDecodeError:
            return f"Error: Could not parse input as JSON. Received: {day_plan}"

    # Verify we now have a dictionary
    if not isinstance(day_plan, dict):
        return f"Error: Expected a dictionary, got {type(day_plan).__name__}"

    # Process the dictionary
    for day, task in day_plan.items():
        todo += f"\n📅 {day}:\n- {task}"
    return todo


format_as_todo_list = Tool(
    name="format_as_todo_list",
    func=format_as_todo_list_func,
    description="Format a daily plan dictionary into a weekly to-do list. Input must be a dict like {'Monday': 'do X', ...} or a JSON string representing such a dict.",
)

# LLM setup
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You're an assistant helping users break weekly goals into a day-wise todo list.
    
First use the list_week_days tool to create a plan, then pass the ENTIRE RESULT (the complete dictionary) to format_as_todo_list.
Do not modify or truncate the dictionary when passing it from one tool to another.""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Create agent and executor
tools = [list_week_days, format_as_todo_list]

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Run the agent
if __name__ == "__main__":
    user_input = "I want to complete a Python project and work out five times this week."
    result = agent_executor.invoke({"input": user_input})
    print("\n📋 Weekly Planner:\n", result["output"])



> Entering new AgentExecutor chain...

Invoking: `list_week_days` with `{'goal': 'Complete a Python project and work out five times'}`


{'Monday': 'Complete a Python project and work out five times - Step 1', 'Tuesday': 'Complete a Python project and work out five times - Step 2', 'Wednesday': 'Complete a Python project and work out five times - Step 3', 'Thursday': 'Complete a Python project and work out five times - Step 4', 'Friday': 'Complete a Python project and work out five times - Step 5', 'Saturday': 'Complete a Python project and work out five times - Step 6', 'Sunday': 'Complete a Python project and work out five times - Step 7'}
Invoking: `format_as_todo_list` with `{"Monday": "Complete a Python project and work out five times - Step 1", "Tuesday": "Complete a Python project and work out five times - Step 2", "Wednesday": "Complete a Python project and work out five times - Step 3", "Thursday": "Complete a Python project and work out five times - Step 4", "Friday": "Comp

In [ ]:
from typing import Dict, List, Optional, Any
import os
import json
from enum import Enum
from dotenv import load_dotenv

# LangChain imports
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory

"""
Weekly Task Planner Agent using LangChain v0.3.24

This script creates an agentic weekly task planner that can:
1. Add tasks to specific days of the week
2. Prioritize tasks based on importance and urgency
3. Suggest optimal scheduling based on task dependencies
4. Reschedule tasks when needed
5. Provide a summary of the week's tasks
"""

load_dotenv()


# Define task priority as an Enum
class Priority(str, Enum):
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"


# Define days of the week
DAYS_OF_WEEK = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]


# Task class to store task information
class Task:
    def __init__(
        self,
        title: str,
        description: str = "",
        day: str = None,
        priority: Priority = Priority.MEDIUM,
        duration_minutes: int = 30,
        dependencies: List[str] = None,
        completed: bool = False,
    ):
        self.title = title
        self.description = description
        self.day = day
        self.priority = priority
        self.duration_minutes = duration_minutes
        self.dependencies = dependencies or []
        self.completed = completed

    def to_dict(self) -> Dict[str, Any]:
        return {
            "title": self.title,
            "description": self.description,
            "day": self.day,
            "priority": self.priority,
            "duration_minutes": self.duration_minutes,
            "dependencies": self.dependencies,
            "completed": self.completed,
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "Task":
        return cls(
            title=data["title"],
            description=data.get("description", ""),
            day=data.get("day"),
            priority=data.get("priority", Priority.MEDIUM),
            duration_minutes=data.get("duration_minutes", 30),
            dependencies=data.get("dependencies", []),
            completed=data.get("completed", False),
        )


# TaskPlanner class to manage the weekly task list
class TaskPlanner:
    def __init__(self, storage_path: str = "weekly_tasks.json"):
        self.storage_path = storage_path
        self.tasks = {}
        self.load_tasks()

    def load_tasks(self) -> None:
        """Load tasks from storage file if it exists."""
        if os.path.exists(self.storage_path):
            try:
                with open(self.storage_path, "r") as f:
                    data = json.load(f)
                    self.tasks = {task_id: Task.from_dict(task_data) for task_id, task_data in data.items()}
            except Exception as e:
                print(f"Error loading tasks: {e}")
                self.tasks = {}

    def save_tasks(self) -> None:
        """Save tasks to storage file."""
        try:
            with open(self.storage_path, "w") as f:
                json.dump({task_id: task.to_dict() for task_id, task in self.tasks.items()}, f, indent=2)
        except Exception as e:
            print(f"Error saving tasks: {e}")

    def add_task(
        self,
        title: str,
        description: str = "",
        day: str = None,
        priority: Priority = Priority.MEDIUM,
        duration_minutes: int = 30,
        dependencies: List[str] = None,
    ) -> str:
        """Add a new task and return its ID."""
        task_id = f"task_{len(self.tasks) + 1}"
        self.tasks[task_id] = Task(
            title=title,
            description=description,
            day=day,
            priority=priority,
            duration_minutes=duration_minutes,
            dependencies=dependencies or [],
        )
        self.save_tasks()
        return task_id

    def update_task(
        self,
        task_id: str,
        title: Optional[str] = None,
        description: Optional[str] = None,
        day: Optional[str] = None,
        priority: Optional[Priority] = None,
        duration_minutes: Optional[int] = None,
        dependencies: Optional[List[str]] = None,
        completed: Optional[bool] = None,
    ) -> bool:
        """Update an existing task."""
        if task_id not in self.tasks:
            return False

        task = self.tasks[task_id]
        if title is not None:
            task.title = title
        if description is not None:
            task.description = description
        if day is not None:
            task.day = day
        if priority is not None:
            task.priority = priority
        if duration_minutes is not None:
            task.duration_minutes = duration_minutes
        if dependencies is not None:
            task.dependencies = dependencies
        if completed is not None:
            task.completed = completed

        self.save_tasks()
        return True

    def delete_task(self, task_id: str) -> bool:
        """Delete a task by ID."""
        if task_id in self.tasks:
            del self.tasks[task_id]
            self.save_tasks()
            return True
        return False

    def complete_task(self, task_id: str) -> bool:
        """Mark a task as completed."""
        if task_id in self.tasks:
            self.tasks[task_id].completed = True
            self.save_tasks()
            return True
        return False

    def get_tasks_by_day(self, day: str) -> Dict[str, Task]:
        """Get all tasks for a specific day."""
        if day not in DAYS_OF_WEEK:
            raise ValueError(f"Invalid day: {day}. Must be one of {DAYS_OF_WEEK}")

        return {task_id: task for task_id, task in self.tasks.items() if task.day == day}

    def get_all_tasks(self) -> Dict[str, Task]:
        """Get all tasks."""
        return self.tasks

    def get_task(self, task_id: str) -> Optional[Task]:
        """Get a specific task by ID."""
        return self.tasks.get(task_id)

    def reschedule_task(self, task_id: str, new_day: str) -> bool:
        """Reschedule a task to a different day."""
        if new_day not in DAYS_OF_WEEK:
            raise ValueError(f"Invalid day: {new_day}. Must be one of {DAYS_OF_WEEK}")

        if task_id in self.tasks:
            self.tasks[task_id].day = new_day
            self.save_tasks()
            return True
        return False

    def get_weekly_summary(self) -> Dict[str, List[Dict[str, Any]]]:
        """Get a summary of tasks organized by day."""
        summary = {day: [] for day in DAYS_OF_WEEK}
        unscheduled = []

        for task_id, task in self.tasks.items():
            task_info = {
                "id": task_id,
                "title": task.title,
                "priority": task.priority,
                "duration": task.duration_minutes,
                "completed": task.completed,
            }

            if task.day in DAYS_OF_WEEK:
                summary[task.day].append(task_info)
            else:
                unscheduled.append(task_info)

        # Add unscheduled tasks
        summary["Unscheduled"] = unscheduled
        return summary


# Define LangChain tools using the TaskPlanner
task_planner = TaskPlanner()


@tool
def add_task(
    title: str,
    description: str = "",
    day: str = None,
    priority: str = "medium",
    duration_minutes: int = 30,
    dependencies: List[str] = None,
) -> str:
    """
    Add a new task to the weekly planner.

    Args:
        title: The title of the task
        description: A detailed description of the task
        day: The day of the week for the task (Monday, Tuesday, etc.)
        priority: The priority level (high, medium, low)
        duration_minutes: Estimated duration in minutes
        dependencies: List of task IDs this task depends on

    Returns:
        The ID of the newly created task
    """
    try:
        if day and day not in DAYS_OF_WEEK:
            return f"Error: Invalid day '{day}'. Must be one of {DAYS_OF_WEEK}"

        if priority not in [p.value for p in Priority]:
            return f"Error: Invalid priority '{priority}'. Must be one of {[p.value for p in Priority]}"

        task_id = task_planner.add_task(
            title=title,
            description=description,
            day=day,
            priority=Priority(priority),
            duration_minutes=duration_minutes,
            dependencies=dependencies or [],
        )
        return f"Task added successfully with ID: {task_id}"
    except Exception as e:
        return f"Error adding task: {str(e)}"


@tool
def update_task(
    task_id: str,
    title: str = None,
    description: str = None,
    day: str = None,
    priority: str = None,
    duration_minutes: int = None,
    dependencies: List[str] = None,
    completed: bool = None,
) -> str:
    """
    Update an existing task in the weekly planner.

    Args:
        task_id: The ID of the task to update
        title: The new title (optional)
        description: The new description (optional)
        day: The new day of the week (optional)
        priority: The new priority level (optional)
        duration_minutes: The new estimated duration in minutes (optional)
        dependencies: The new list of dependencies (optional)
        completed: Whether the task is completed (optional)

    Returns:
        Success or error message
    """
    try:
        if day and day not in DAYS_OF_WEEK:
            return f"Error: Invalid day '{day}'. Must be one of {DAYS_OF_WEEK}"

        if priority and priority not in [p.value for p in Priority]:
            return f"Error: Invalid priority '{priority}'. Must be one of {[p.value for p in Priority]}"

        priority_enum = Priority(priority) if priority else None

        if task_planner.update_task(
            task_id=task_id,
            title=title,
            description=description,
            day=day,
            priority=priority_enum,
            duration_minutes=duration_minutes,
            dependencies=dependencies,
            completed=completed,
        ):
            return f"Task {task_id} updated successfully"
        else:
            return f"Error: Task {task_id} not found"
    except Exception as e:
        return f"Error updating task: {str(e)}"


@tool
def delete_task(task_id: str) -> str:
    """
    Delete a task from the weekly planner.

    Args:
        task_id: The ID of the task to delete

    Returns:
        Success or error message
    """
    if task_planner.delete_task(task_id):
        return f"Task {task_id} deleted successfully"
    else:
        return f"Error: Task {task_id} not found"


@tool
def complete_task(task_id: str) -> str:
    """
    Mark a task as completed.

    Args:
        task_id: The ID of the task to mark as completed

    Returns:
        Success or error message
    """
    if task_planner.complete_task(task_id):
        return f"Task {task_id} marked as completed"
    else:
        return f"Error: Task {task_id} not found"


@tool
def get_tasks_for_day(day: str) -> str:
    """
    Get all tasks scheduled for a specific day.

    Args:
        day: The day of the week (Monday, Tuesday, etc.)

    Returns:
        JSON string containing tasks for the specified day
    """
    try:
        if day not in DAYS_OF_WEEK:
            return f"Error: Invalid day '{day}'. Must be one of {DAYS_OF_WEEK}"

        tasks = task_planner.get_tasks_by_day(day)
        result = {}

        for task_id, task in tasks.items():
            result[task_id] = {
                "title": task.title,
                "description": task.description,
                "priority": task.priority,
                "duration_minutes": task.duration_minutes,
                "completed": task.completed,
            }

        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error getting tasks for day: {str(e)}"


@tool
def get_all_tasks() -> str:
    """
    Get all tasks in the weekly planner.

    Returns:
        JSON string containing all tasks
    """
    try:
        tasks = task_planner.get_all_tasks()
        result = {}

        for task_id, task in tasks.items():
            result[task_id] = {
                "title": task.title,
                "description": task.description,
                "day": task.day,
                "priority": task.priority,
                "duration_minutes": task.duration_minutes,
                "dependencies": task.dependencies,
                "completed": task.completed,
            }

        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error getting all tasks: {str(e)}"


@tool
def get_weekly_summary() -> str:
    """
    Get a summary of all tasks organized by day.

    Returns:
        A formatted summary of the week's tasks
    """
    try:
        summary = task_planner.get_weekly_summary()
        result = {}

        for day, tasks in summary.items():
            result[day] = tasks

        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error getting weekly summary: {str(e)}"


@tool
def suggest_optimal_schedule() -> str:
    """
    Suggest an optimal schedule based on task priorities, durations, and dependencies.

    Returns:
        A suggested optimal schedule
    """
    try:
        tasks = task_planner.get_all_tasks()

        # Convert tasks to a format easier for scheduling
        task_list = []
        dependencies_map = {}

        for task_id, task in tasks.items():
            task_list.append(
                {
                    "id": task_id,
                    "title": task.title,
                    "priority": task.priority,
                    "duration": task.duration_minutes,
                    "dependencies": task.dependencies,
                    "completed": task.completed,
                    "current_day": task.day,
                }
            )

            # Build dependency map
            for dep_id in task.dependencies:
                if dep_id in dependencies_map:
                    dependencies_map[dep_id].append(task_id)
                else:
                    dependencies_map[dep_id] = [task_id]

        # Sort tasks by priority and dependencies
        priority_values = {Priority.HIGH: 3, Priority.MEDIUM: 2, Priority.LOW: 1}

        # First, sort by completed status (incomplete first)
        task_list.sort(key=lambda x: x["completed"])

        # Then sort by priority
        task_list.sort(key=lambda x: priority_values.get(x["priority"], 0), reverse=True)

        # Initialize schedule
        schedule = {day: [] for day in DAYS_OF_WEEK}
        day_loads = {day: 0 for day in DAYS_OF_WEEK}  # Minutes allocated per day

        # Place tasks with dependencies first
        placed_tasks = set()
        unplaced_tasks = []

        for task in task_list:
            if task["completed"]:
                placed_tasks.add(task["id"])
                if task["current_day"]:
                    schedule[task["current_day"]].append(task)
                continue

            if not task["dependencies"]:
                unplaced_tasks.append(task)
                continue

            # Check if all dependencies are satisfied
            all_deps_placed = all(dep_id in placed_tasks for dep_id in task["dependencies"])

            if all_deps_placed:
                # Find the best day for this task
                best_day = min(DAYS_OF_WEEK, key=lambda d: day_loads[d])
                schedule[best_day].append(task)
                day_loads[best_day] += task["duration"]
                placed_tasks.add(task["id"])
            else:
                unplaced_tasks.append(task)

        # Place remaining tasks
        for task in unplaced_tasks:
            if task["completed"] or task["id"] in placed_tasks:
                continue

            # Find the best day for this task
            best_day = min(DAYS_OF_WEEK, key=lambda d: day_loads[d])
            schedule[best_day].append(task)
            day_loads[best_day] += task["duration"]
            placed_tasks.add(task["id"])

        # Format the result
        result = {}
        for day, tasks in schedule.items():
            if tasks:
                result[day] = [
                    {
                        "id": task["id"],
                        "title": task["title"],
                        "priority": task["priority"],
                        "duration_minutes": task["duration"],
                    }
                    for task in tasks
                ]
            else:
                result[day] = []

        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error suggesting optimal schedule: {str(e)}"


@tool
def reschedule_task(task_id: str, new_day: str) -> str:
    """
    Reschedule a task to a different day.

    Args:
        task_id: The ID of the task to reschedule
        new_day: The new day of the week for the task

    Returns:
        Success or error message
    """
    try:
        if new_day not in DAYS_OF_WEEK:
            return f"Error: Invalid day '{new_day}'. Must be one of {DAYS_OF_WEEK}"

        if task_planner.reschedule_task(task_id, new_day):
            return f"Task {task_id} rescheduled to {new_day} successfully"
        else:
            return f"Error: Task {task_id} not found"
    except Exception as e:
        return f"Error rescheduling task: {str(e)}"


# Set up the LangChain agent
tools = [
    add_task,
    update_task,
    delete_task,
    complete_task,
    get_tasks_for_day,
    get_all_tasks,
    get_weekly_summary,
    suggest_optimal_schedule,
    reschedule_task,
]

# System message for the agent
system_message = """
You are a helpful Weekly Task Planner assistant. Your job is to help the user manage their weekly tasks.
You can add tasks, update tasks, delete tasks, mark tasks as completed, and get information about tasks.
You can also suggest an optimal schedule based on task priorities, durations, and dependencies.

Always be concise and helpful. When the user asks you to do something with tasks, use the appropriate tool.
When dealing with days, always use the format: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday.

For priorities, use: high, medium, or low.
"""

# Set up the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Initialize the LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)


# Function to run the agent
def run_task_planner_agent():
    print("Weekly Task Planner Agent")
    print("------------------------")
    print("Type 'quit' or 'exit' to end the conversation.")
    print()

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["quit", "exit", ""]:
            print("Task Planner: Goodbye!")
            break

        try:
            response = agent_executor.invoke({"input": user_input})
            print(f"Task Planner: {response['output']}")
        except Exception as e:
            print(f"Task Planner: I encountered an error: {str(e)}")


if __name__ == "__main__":
    run_task_planner_agent()